In [1]:
import os
import re
import pandas as pd
import glob
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment

def parse_results_from_text(text):
    """从txt文本中解析模型结果"""
    
    # 首先提取摘要信息
    summary_pattern = r'# (\w+) 模型归一化后的系数 \(按绝对值排序\)\s*\n\s*-+\s*\n(总计 \d+ 个有效变量.*?)\s*\n\s*-+\s*\n'
    summary_match = re.search(summary_pattern, text, re.DOTALL)
    
    model_type = None
    summary_text = None
    has_significant_vars = False
    
    if summary_match:
        model_type = summary_match.group(1)  # OLS, TOBIT等
        summary_text = summary_match.group(2)  # "总计 X 个有效变量, 显著 (p<0.05)" 或 "总计 X 个有效变量, 无显著变量"
        has_significant_vars = "显著" in summary_text and "无显著变量" not in summary_text
        
        print(f"找到{model_type}模型摘要: {summary_text}")
        print(f"是否有显著变量: {has_significant_vars}")
    else:
        print("未找到模型摘要信息")
        return None, {"error": "未找到模型摘要信息"}, None
    
    # 如果没有显著变量，直接返回摘要信息
    model_data = []
    model_info = {
        "model_type": model_type,
        "summary": summary_text,
        "has_significant_vars": has_significant_vars
    }
    
    # 只有在有显著变量时才继续解析表格
    if has_significant_vars:
        # 提取表格内容
        table_pattern = r'\| 变量 \| 变量类型 \| 系数 \| p值 \| 归一化系数 \| 归一化百分比 \| 显著性 \|\s*\n\|[-\s|]+\|\s*\n((?:\|.*?\|\s*\n)+)'
        table_match = re.search(table_pattern, text, re.DOTALL)
        
        if table_match:
            table_content = table_match.group(1)
            rows = table_content.strip().split('\n')
            
            for row in rows:
                cells = [cell.strip() for cell in row.split('|')]
                if len(cells) >= 8:  # 包含前后的空单元格，所以>=8
                    var_name = cells[1].strip()
                    var_type = cells[2].strip()
                    coef = cells[3].strip()
                    p_value = cells[4].strip()
                    norm_coef = cells[5].strip()
                    norm_percent = cells[6].strip()
                    significant = cells[7].strip()
                    
                    # 只保留显著的变量
                    if significant.lower() == '是':
                        try:
                            model_data.append({
                                "变量": var_name,
                                "变量类型": var_type,
                                "系数": float(coef) if coef.lower() != 'nan' else None,
                                "p值": float(p_value) if p_value.lower() != 'nan' else None,
                                "归一化系数": float(norm_coef) if norm_coef.lower() != 'nan' else None,
                                "归一化百分比": norm_percent,
                                "显著性": significant
                            })
                        except ValueError:
                            print(f"警告: 无法解析数值 - {row}")
            
            print(f"从表格中提取了 {len(model_data)} 个显著变量")
        else:
            print("警告: 找到摘要但未找到表格内容")
    
    # 尝试提取VIF信息（如果有的话）
    vif_pattern = r'\| 变量 \| VIF \|\s*\n\|[-]+\|[-]+\|\s*\n((?:\|[^|]+\|[^|]+\|\s*\n)+)'
    vif_match = re.search(vif_pattern, text, re.DOTALL)
    
    vif_data = []
    if vif_match:
        vif_content = vif_match.group(1)
        if vif_content:
            vif_rows = vif_content.strip().split('\n')
            for row in vif_rows:
                parts = row.split('|')
                if len(parts) >= 3:  # 应该至少有3部分：空、变量名、VIF值、空
                    var_name = parts[1].strip()
                    vif_value = parts[2].strip()
                    if var_name and vif_value:
                        vif_data.append([var_name, vif_value])
        print(f"提取了 {len(vif_data)} 个变量的VIF值")
    
    return model_data, model_info, vif_data

def export_to_excel(model_data, model_info, vif_data, output_path):
    """将解析的数据导出到Excel文件"""
    # 创建Excel工作簿
    wb = Workbook()
    
    # 回归结果工作表
    ws_model = wb.active
    ws_model.title = "回归结果"
    
    # 获取模型类型和摘要
    model_type = model_info.get("model_type", "未知")
    summary_text = model_info.get("summary", "")
    has_significant_vars = model_info.get("has_significant_vars", False)
    
    # 添加标题和摘要
    ws_model.append([f"# {model_type} 模型归一化后的系数 (按绝对值排序)"])
    ws_model.append([])
    ws_model.append(["-----------------------------------"])
    ws_model.append([summary_text])
    ws_model.append(["-----------------------------------"])
    
    # 如果有显著变量，添加表格
    if has_significant_vars and model_data:
        ws_model.append([])
        
        # 添加表头
        ws_model.append(["变量", "变量类型", "系数", "p值", "归一化系数", "归一化百分比", "显著性"])
        
        # 添加数据行
        for item in model_data:
            ws_model.append([
                item["变量"],
                item["变量类型"],
                item["系数"],
                item["p值"],
                item["归一化系数"],
                item["归一化百分比"],
                item["显著性"]
            ])
    
    # VIF工作表（如果有VIF数据）
    if vif_data:
        ws_vif = wb.create_sheet("多重共线性检验")
        ws_vif.append(["变量VIF值"])
        ws_vif.append([])
        
        # 添加表头和数据
        ws_vif.append(["变量", "VIF"])
        for var_name, vif_value in vif_data:
            ws_vif.append([var_name, vif_value])
    
    # 美化工作表
    for ws in wb.worksheets:
        # 调整列宽
        for column in ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                if cell.value:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
            adjusted_width = (max_length + 2)
            ws.column_dimensions[column_letter].width = adjusted_width
        
        # 设置标题样式
        ws['A1'].font = Font(bold=True, size=14)
        
        # 设置表头样式
        if ws.title == "回归结果" and has_significant_vars:
            header_row = 7  # 在回归结果表中，表头在第7行
        else:
            header_row = 3  # 在其他表中，表头在第3行
            
        for cell in ws[header_row]:
            if cell.value:
                cell.font = Font(bold=True)
                cell.alignment = Alignment(horizontal='center')
    
    # 保存工作簿
    wb.save(output_path)
    print(f"已保存Excel文件: {output_path}")

def process_all_files():
    """处理子文件夹中所有符合模式的文件"""
    # 指定子文件夹
    subfolder = "生成结果/social_media/"
    
    # 确保子文件夹存在
    if not os.path.exists(subfolder):
        print(f"子文件夹 '{subfolder}' 不存在！")
        return
    
    # 构建文件路径模式
    file_pattern = os.path.join(subfolder, "*-归一化的模型结果.txt")
    result_files = glob.glob(file_pattern)
    
    if not result_files:
        print(f"没有找到符合模式'{file_pattern}'的文件")
        return
    
    print(f"在'{subfolder}'中找到 {len(result_files)} 个文件待处理")
    
    # 处理每个文件
    for file_path in result_files:
        # 提取文件名前缀
        prefix = os.path.basename(file_path).split('-')[0]
        print(f"\n正在处理 {file_path} (前缀: {prefix})...")
        
        try:
            # 读取文件内容
            with open(file_path, 'r', encoding='utf-8') as file:
                result_text = file.read()
            
            # 检查文件内容
            if len(result_text) < 100:
                print(f"警告: 文件内容太短 ({len(result_text)} 字符)")
                # 生成一个基本的Excel文件
                model_data = []
                model_info = {
                    "model_type": "未知",
                    "summary": "文件内容太短，无法提取有效信息",
                    "has_significant_vars": False
                }
                vif_data = []
                
                output_path = os.path.join(subfolder, f"{prefix}-模型结果.xlsx")
                export_to_excel(model_data, model_info, vif_data, output_path)
                continue
            
            # 打印文件内容的一部分用于调试
            preview = result_text[:200].replace('\n', ' ')
            print(f"文件内容预览 (前200字符): {preview}...")
            
            # 解析文本内容
            try:
                model_data, model_info, vif_data = parse_results_from_text(result_text)
                
                if model_info.get("error"):
                    print(f"警告: {model_info['error']}")
                    # 尝试备用解析方法
                    print("尝试备用解析方法...")
                    summary_pattern = r'总计 (\d+) 个.*?变量.*?(无显著变量|显著 \(p<0\.05\))'
                    summary_match = re.search(summary_pattern, result_text)
                    
                    if summary_match:
                        var_count = summary_match.group(1)
                        sig_status = summary_match.group(2)
                        model_type = "未知"
                        if "OLS" in result_text:
                            model_type = "OLS"
                        elif "TOBIT" in result_text:
                            model_type = "TOBIT"
                        
                        has_significant_vars = sig_status != "无显著变量"
                        summary_text = f"总计 {var_count} 个有效变量, {sig_status}"
                        
                        model_info = {
                            "model_type": model_type,
                            "summary": summary_text,
                            "has_significant_vars": has_significant_vars
                        }
                        
                        # 如果有显著变量，尝试提取
                        if has_significant_vars:
                            # 简化的表格提取
                            rows = re.findall(r'\|([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)\|([^|]+)\|', result_text)
                            for row in rows:
                                if '是' in row[6]:  # 显著性为"是"
                                    try:
                                        model_data.append({
                                            "变量": row[0].strip(),
                                            "变量类型": row[1].strip(),
                                            "系数": float(row[2].strip()) if row[2].strip().lower() != 'nan' else None,
                                            "p值": float(row[3].strip()) if row[3].strip().lower() != 'nan' else None,
                                            "归一化系数": float(row[4].strip()) if row[4].strip().lower() != 'nan' else None,
                                            "归一化百分比": row[5].strip(),
                                            "显著性": row[6].strip()
                                        })
                                    except ValueError:
                                        pass
                    else:
                        # 仍然无法解析，使用默认信息
                        model_type = "未知"
                        if "OLS" in result_text:
                            model_type = "OLS"
                        elif "TOBIT" in result_text:
                            model_type = "TOBIT"
                            
                        model_info = {
                            "model_type": model_type,
                            "summary": "无法解析摘要信息",
                            "has_significant_vars": False
                        }
                
                # 输出Excel文件路径（保存到同一子文件夹）
                output_path = os.path.join(subfolder, f"{prefix}-模型结果.xlsx")
                
                # 导出到Excel
                export_to_excel(model_data, model_info, vif_data, output_path)
                
            except Exception as e:
                print(f"解析过程出错: {str(e)}")
                import traceback
                print(traceback.format_exc())
                
                # 创建一个基本的Excel文件
                model_data = []
                model_info = {
                    "model_type": "未知",
                    "summary": f"解析过程出错: {str(e)}",
                    "has_significant_vars": False
                }
                vif_data = []
                
                output_path = os.path.join(subfolder, f"{prefix}-模型结果.xlsx")
                export_to_excel(model_data, model_info, vif_data, output_path)
                print(f"✓ 创建了错误报告Excel文件: {output_path}")
        
        except Exception as e:
            import traceback
            print(f"✗ 处理 {file_path} 时出错: {str(e)}")
            print(traceback.format_exc())
            
            # 创建一个错误报告Excel文件
            try:
                model_data = []
                model_info = {
                    "model_type": "未知",
                    "summary": f"文件处理出错: {str(e)}",
                    "has_significant_vars": False
                }
                vif_data = []
                
                output_path = os.path.join(subfolder, f"{prefix}-模型结果.xlsx")
                export_to_excel(model_data, model_info, vif_data, output_path)
                print(f"✓ 创建了错误报告Excel文件: {output_path}")
            except:
                print("无法创建错误报告Excel文件")

# 直接执行函数，适合在Jupyter Notebook中运行
process_all_files()

在'生成结果/social_media/'中找到 5 个文件待处理

正在处理 生成结果/social_media\Comment-归一化的模型结果.txt (前缀: Comment)...
文件内容预览 (前200字符): # TOBIT 模型归一化后的系数 (按绝对值排序)  ----------------------------------- 总计 20 个有效变量, 无显著变量 -----------------------------------  | 变量 | 变量类型 | 系数 | p值 | 归一化系数 | 归一化百分比 | 显著性 | |------|---------|------|------|-...
找到TOBIT模型摘要: 总计 20 个有效变量, 无显著变量
是否有显著变量: False
已保存Excel文件: 生成结果/social_media/Comment-模型结果.xlsx

正在处理 生成结果/social_media\Like-归一化的模型结果.txt (前缀: Like)...
文件内容预览 (前200字符): # TOBIT 模型归一化后的系数 (按绝对值排序)  ----------------------------------- 总计 20 个有效变量, 无显著变量 -----------------------------------  | 变量 | 变量类型 | 系数 | p值 | 归一化系数 | 归一化百分比 | 显著性 | |------|---------|------|------|-...
找到TOBIT模型摘要: 总计 20 个有效变量, 无显著变量
是否有显著变量: False
已保存Excel文件: 生成结果/social_media/Like-模型结果.xlsx

正在处理 生成结果/social_media\Repost-归一化的模型结果.txt (前缀: Repost)...
文件内容预览 (前200字符): # TOBIT 模型归一化后的系数 (按绝对值排序)  ----------------------------------- 总计 20 个有效变量, 无显著变量 -----------------------------------  | 变量 | 变量类型 | 系数 | p值 